In [1]:
import sys
import os
os.chdir("/home/jonfrey/ASL")
sys.path.append("""/home/jonfrey/ASL/src/""")
sys.path.append("""/home/jonfrey/ASL/src/pseudo_label""")

from pseudo_label.yolo import YoloHelper
from pseudo_label.deeplab import DeeplabHelper
from pseudo_label.fast_scnn import FastSCNNHelper

import numpy as np
from visu import Visualizer
import imageio

visu = Visualizer(os.getenv('HOME')+'/tmp', logger=None, epoch=0, store=False, num_classes=41)

yh = YoloHelper()
dlh = DeeplabHelper(device="cuda:1")
fsh = FastSCNNHelper(device='cuda:1')

Using cache found in /home/jonfrey/.cache/torch/hub/ultralytics_yolov5_master

                 from  n    params  module                                  arguments                     
  0                -1  1      8800  models.common.Focus                     [3, 80, 3]                    
  1                -1  1    115520  models.common.Conv                      [80, 160, 3, 2]               
  2                -1  1    309120  models.common.C3                        [160, 160, 4]                 
  3                -1  1    461440  models.common.Conv                      [160, 320, 3, 2]              
  4                -1  1   3285760  models.common.C3                        [320, 320, 12]                
  5                -1  1   1844480  models.common.Conv                      [320, 640, 3, 2]              
  6                -1  1  13125120  models.common.C3                        [640, 640, 12]                
  7                -1  1   7375360  models.common.Conv           

Adding autoShape... 


Using cache found in /home/jonfrey/.cache/torch/hub/kazuto1011_deeplab-pytorch_master


Restoring weights: <All keys matched successfully>


In [9]:
from datasets_asl import get_dataset
DEVICE = 'cuda:1'

import yaml
import coloredlogs
coloredlogs.install()
import time
import argparse

from pathlib import Path
import gc
import torch

from utils_asl import load_yaml

env_cfg_path = os.path.join('cfg/env', os.environ['ENV_WORKSTATION_NAME']+ '.yml')
env_cfg = load_yaml(env_cfg_path)
eval_cfg = load_yaml("/home/jonfrey/ASL/cfg/eval/eval.yml")

# SETUP DATALOADER
dataset_test = get_dataset(
    **eval_cfg['dataset'],
    env = env_cfg,
    output_trafo = None,
    )
dataloader_test = torch.utils.data.DataLoader(dataset_test,
    shuffle = True,
    num_workers = 0,
    pin_memory = eval_cfg['loader']['pin_memory'],
    batch_size = 1, 
    drop_last = True)

st = time.time()
globale_idx_to_image_path = dataset_test.image_pths

In [3]:
def get_max_acc(label, gt , names = ['deeplab', 'yolo', 'fastscnn', 'gt'] ):
    m = gt != -1
    acc_indi = {}
    for l,n in zip(label,names):
        correct = ( l[m] == gt[m]).sum()
        total = m.sum()
        acc_indi[n] = correct/total
        
    # Optimal upper bound
    m = gt != -1
    m_correct = np.zeros( label[0].shape )
    for l,n in zip(label,names):
        m_est = l == gt
        m_correct[m_est] = 1
    acc_upper_bound = m_correct[m].sum() / m.sum()
    acc_indi['upper_bound'] = acc_upper_bound
    
    
    # Create one hot valume NR_Labels,H,W,NR_CLASSES
    onehot = [torch.nn.functional.one_hot( torch.from_numpy(l+1).type(torch.int64), num_classes=41)[None,:,:,1:] for l in label ] 
    onehot[2] = onehot[2]+0.001 # Set FASTSCNN to be dominant
    s = torch.cat( onehot, dim=0).sum(dim=0) # Sum oder the labels
    label_pred = torch.argmax(s, dim=2) # get Onehot index with heighest probability
    agreement_fusion = (label_pred.numpy() == gt)[m].sum() / m.sum()
    acc_indi['agreement_fusion'] = agreement_fusion
    
    return acc_indi

In [ ]:
from pseudo_label import readSegmentation
import time

st = time.time()
counts = 0
length = len(dataloader_test)
for j, batch in enumerate( dataloader_test ):
    # START EVALUATION  
    images = batch[0]
    target = batch[1]
    ori_img = batch[2]
    replayed = batch[3]
    BS = images.shape[0]
    global_idx = batch[4] 
    
    images *= 255
    images = images.permute(0,2,3,1).numpy().astype(np.uint8)
    
    for b in range( images.shape[0] ):
#         print("Deeplab v3")
        label_dl = dlh.get_label( images[b] )
#         visu.plot_segmentation(seg=label_dl+1,jupyter=True, method='right')
#         visu.plot_image(img=images[0],jupyter=True, method='left')
#         print("YOLO v5")
        label_yolo = yh.get_label( images[b] )
#         visu.plot_segmentation(seg=label_yolo+1,jupyter=True, method='right')
#         visu.plot_image(img=images[0],jupyter=True, method='left')
        label_fastscnn = fsh.get_label( images[b] )
#         print("FastSCNN")
#         visu.plot_segmentation(seg=label_fastscnn+1,jupyter=True, method='right')
#         visu.plot_image(img=images[0],jupyter=True, method='left')
#         print("GT")
#         visu.plot_segmentation(seg=target+1,jupyter=True, method='right')
#         visu.plot_image(img=images[0],jupyter=True, method='left')
        
        ret = get_max_acc(
                label=[label_dl, label_yolo, label_fastscnn], 
                gt=target[b].numpy(), 
                names= ['deeplab', 'yolo', 'fastscnn'])
        
        if counts == 0:
            acc_dict = ret
        else:
            for k in ret.keys():
                acc_dict[k] += ret[k]
                
        counts += 1
    
    if j % 10 == 0:
        mini =int((time.time()-st)/60)
        mini_left =int ((time.time()-st)/j*(length-j)/60)
        print(f'{j}/{length} total time elapsed: {mini}min; time left {mini_left}min')
    if j > 500: 
        break


for k in acc_dict.keys():
    acc_dict[k] /= counts

print(acc_dict)

/home/jonfrey/miniconda3/envs/track4/lib/python3.7/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


0/6851 total time elapsed: 0min
10/6851 total time elapsed: 0min
20/6851 total time elapsed: 0min
30/6851 total time elapsed: 0min
40/6851 total time elapsed: 0min
50/6851 total time elapsed: 0min
60/6851 total time elapsed: 1min
70/6851 total time elapsed: 1min
80/6851 total time elapsed: 1min
90/6851 total time elapsed: 1min
100/6851 total time elapsed: 1min
110/6851 total time elapsed: 2min
120/6851 total time elapsed: 2min
130/6851 total time elapsed: 2min
140/6851 total time elapsed: 2min
150/6851 total time elapsed: 2min
160/6851 total time elapsed: 2min
170/6851 total time elapsed: 3min
180/6851 total time elapsed: 3min
190/6851 total time elapsed: 3min
200/6851 total time elapsed: 3min
210/6851 total time elapsed: 3min
220/6851 total time elapsed: 4min
230/6851 total time elapsed: 4min
240/6851 total time elapsed: 4min
250/6851 total time elapsed: 4min
260/6851 total time elapsed: 4min
270/6851 total time elapsed: 4min
280/6851 total time elapsed: 5min
290/6851 total time elaps